In [1]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns, Distributions, Clustering, AutoGrad
using PyCall
#using PlotRecipes
#using TestImages, Images
import SumProductNetworks.add!

#### include local code

In [46]:
include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")
include("regionGraph.jl")
include("regionGraphUtils.jl")
include("gpUtils.jl")

mcmcKernel (generic function with 1 method)

# Load data

In [3]:
datapath = "../data/clean/motor.csv"
(data, header) = readcsv(datapath, header = true)
headerDict = Dict(col[2] => col[1] for col in enumerate(header))
X = convert(Vector, data[:,headerDict["times"]])
y = convert(Vector, data[:,headerDict["accel"]]);
y /= maximum(y);

In [4]:
?GaussianProcesses.optimize!

No documentation found.

`GaussianProcesses.optimize!` is a `Function`.

```
# 1 method for generic function "optimize!":
optimize!(gp::GaussianProcesses.GPBase; method, mean, kern, noise, lik, kwargs...) in GaussianProcesses at /Users/martin/.julia/v0.6/GaussianProcesses/src/optimize.jl:20
```


In [5]:
S = 100
kernel_function = SE(log(5.0),log(1.0))
kernel_function = Mat12Iso(log(5.0), log(1.0))
gp = GP(reshape(X, 1, length(X)), y, MeanZero(), kernel_function, -1.)
GaussianProcesses.optimize!(gp, mean = false, kern = true, noise = false, lik=false)
μgp, σgp = predict_y(gp,linspace(minimum(X),maximum(X),S));

scatter(X, y, label = "observations")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp + 2*σgp, color = :lightgreen, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp - 2*σgp, color = :lightgreen, label = "Full GP mean - 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp, color = :orange, label = "Full GP mean")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3402)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 54.4598,102.526 59.6167,103.932 64.7737,104.736 69.9306,103.892 75.0876,101.319 80.2446,100.856 85.4015,102.496 90.5585,105.524 95.7154,106.127 100.872,105.302 
 106.029,106.412 111.186,106.259 116.343,106.424 121.5,108.141 126.657,109.623 131.814,109.126 136.971,108.279 142.128,106.851 147.285,107.523 152.442,110.614 
 157.599,114.842 162.756,120.301 167.913,128.935 173.07,141.178 178.227,155.186 183.384,168.355 188.541,191.198 193.698,208.35 198.855,230.943 204.012,252.042 
 209.169,267.249 214.326,277.729 219.482,284.248 224.639,285.014 229.796,285.595 234.953,283.667 240.11,278.363 245.267,275.54 250.424,266.081 255.581,247.629 
 260.738,222.223 265.895,204.25 271.052,187.417 276.209,179.814 281.366,155.888 286.523,132.954 291.68,113.025 296.837,95.945 301.994,88.7512 307.151,76.9349 
 312.308,62.4866 317.465,51.7947 322.622,48.2921 327.779,43.3289 332.936,43.9884 338.093,48.1478 343.25,52.2041 348.406,55.6461 353.563,55.3781 358.72,62.7653 
 363.877,72.084 369.034,79.1624 374.191,74.7884 379.348,72.5267 384.505,73.3182 389.662,81.6862 394.819,92.518 399.976,98.8974 405.133,100.113 410.29,91.6716 
 415.447,84.5178 420.604,82.1976 425.761,84.4811 430.918,88.2904 436.075,91.149 441.232,94.7288 446.389,94.3189 451.546,93.9011 456.703,94.9748 461.86,98.2098 
 467.017,102.9 472.174,109.56 477.33,111.809 482.487,111.451 487.644,107.747 492.801,104.457 497.958,103.097 503.115,101.307 508.272,99.6414 513.429,99.9768 
 518.586,101.458 523.743,105.08 528.9,102.738 534.057,101.4 539.214,102.04 544.371,101.395 549.528,95.8921 554.685,91.6415 559.842,88.9098 564.999,87.6843 
 
 "/>
<polyline clip-path="url(#clip3402)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 54.4598,187.22 59.6167,186.726 64.7737,186.307 69.9306,187.978 75.0876,190.93 80.2446,191.789 85.4015,190.562 90.5585,187.899 95.7154,187.295 100.872,188.536 
 106.029,187.802 111.186,188.124 116.343,188.555 121.5,187.905 126.657,187.85 131.814,188.846 136.971,189.958 142.128,193.488 147.285,195.339 152.442,194.816 
 157.599,194.837 162.756,201.16 167.913,207.571 173.07,219.404 178.227,230.836 183.384,244.098 188.541,269.506 193.698,288.222 198.855,311.794 204.012,332.304 
 209.169,346.075 214.326,356.236 219.482,362.836 224.639,365.511 229.796,363.995 234.953,362.771 240.11,361.21 245.267,356.86 250.424,345.82 255.581,325.66 
 260.738,300.545 265.895,282.094 271.052,264.833 276.209,256.985 281.366,234.041 286.523,211.645 291.68,192.075 296.837,174.758 301.994,170.929 307.151,160.232 
 312.308,144.944 317.465,133.674 322.622,130.349 327.779,125.592 332.936,127.091 338.093,130.027 343.25,132.703 348.406,136.078 353.563,134.237 358.72,140.712 
 363.877,153.4 369.034,164.613 374.191,164.437 379.348,162.019 384.505,159.699 389.662,168.017 394.819,173.77 399.976,180.652 405.133,182.08 410.29,176.992 
 415.447,168.924 420.604,165.693 425.761,165.3 430.918,169.431 436.075,174.4 441.232,176.799 446.389,178.264 451.546,181.506 456.703,185.126 461.86,186.567 
 467.017,192.207 472.174,198.015 477.33,200.469 482.487,199.848 487.644,199.427 492.801,197.757 497.958,194.201 503.115,192.213 508.272,191.013 513.429,188.18 
 518.586,192.048 523.743,193.796 528.9,194.618 534.057,193.622 539.214,190.672 544.371,188.565 549.528,189.974 554.685,190.066 559.84

In [6]:
function optimize2!(gp::GPBase; method=LBFGS(), mean::Bool=true, kern::Bool=true, noise::Bool=true, lik::Bool=true, kwargs...)
    params_kwargs = GaussianProcesses.get_params_kwargs(typeof(gp); mean=false, kern=true, noise=false, lik=false)
    println(params_kwargs)
        
    func = GaussianProcesses.get_optim_target(gp; params_kwargs...)
    init = GaussianProcesses.get_params(gp; params_kwargs...)  # Initial hyperparameter values
    
    lower = ones(length(init)) * -400
    upper = ones(length(init)) * 400
        
    #results = optimize(od, initial_x, lower, upper, Fminbox{GradientDescent}())
    results = Optim.optimize(func, init, lower, upper, Fminbox{GradientDescent}())     # Run optimizer
    
    min_results = Optim.minimizer(results)
    
    GaussianProcesses.set_params!(gp, min_results; params_kwargs...)
    GaussianProcesses.update_target!(gp)
    
    println(min_results)
    
    println(results)
    
    println("Model parameters:", GaussianProcesses.get_params(gp; params_kwargs...))
    
    return results
end

optimize2! (generic function with 1 method)

In [7]:
kernel_function = LinIso(log(5.0))
gp = GP(reshape(X[s], 1, length(X[s])), y[s], MeanZero(), kernel_function, -1.)
r = optimize2!(gp)
println(r.minimizer)

LoadError: [91mUndefVarError: s not defined[39m

In [156]:
gp.k

Type: GaussianProcesses.LinIso, Params: [Inf]


In [147]:
s = (X .> 40) .& (X .<= 50)
#kernel_function = Mat12Iso(log(5.0), log(1.0))
#gp = GP(reshape(X[s], 1, length(X[s])), y[s], MeanZero(), kernel_function, -1.)
#GaussianProcesses.optimize!(gp, method = GradientDescent(), mean = false, kern = true, noise = false, lik=false)
μgp, σgp = predict_y(gp,linspace(minimum(X[s]),maximum(X[s]),S));

scatter(X[s], y[s], label = "observations")
plot!(linspace(minimum(X[s]),maximum(X[s]),S), w =4, μgp + 2*σgp, color = :lightgreen, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X[s]),maximum(X[s]),S), w =4, μgp - 2*σgp, color = :lightgreen, label = "Full GP mean - 2*std")
plot!(linspace(minimum(X[s]),maximum(X[s]),S), w =4, μgp, color = :orange, label = "Full GP mean")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 42 
 
 
 44 
 
 
 46 
 
 
 48 
 
 
 -0.2 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3302)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 54.4598,84.7099 59.6167,84.7099 64.7737,84.7099 69.9306,84.7099 75.0876,84.7099 80.2446,84.7099 85.4015,84.7099 90.5585,84.7099 95.7154,84.7099 100.872,84.7099 
 106.029,84.7099 111.186,84.7099 116.343,84.7099 121.5,84.7099 126.657,84.7099 131.814,84.7099 136.971,84.7099 142.128,84.7099 147.285,84.7099 152.442,84.7099 
 157.599,84.7099 162.756,84.7099 167.913,84.7099 173.07,84.7099 178.227,84.7099 183.384,84.7099 188.541,84.7099 193.698,84.7099 198.855,84.7099 204.012,84.7099 
 209.169,84.7099 214.326,84.7099 219.482,84.7099 224.639,84.7099 229.796,84.7099 234.953,84.7099 240.11,84.7099 245.267,84.7099 250.424,84.7099 255.581,84.7099 
 260.738,84.7099 265.895,84.7099 271.052,84.7099 276.209,84.7099 281.366,84.7099 286.523,84.7099 291.68,84.7099 296.837,84.7099 301.994,84.7099 307.151,84.7099 
 312.308,84.7099 317.465,84.7099 322.622,84.7099 327.779,84.7099 332.936,84.7099 338.093,84.7099 343.25,84.7099 348.406,84.7099 353.563,84.7099 358.72,84.7099 
 363.877,84.7099 369.034,84.7099 374.191,84.7099 379.348,84.7099 384.505,84.7099 389.662,84.7099 394.819,84.7099 399.976,84.7099 405.133,84.7099 410.29,84.7099 
 415.447,84.7099 420.604,84.7099 425.761,84.7099 430.918,84.7099 436.075,84.7099 441.232,84.7099 446.389,84.7099 451.546,84.7099 456.703,84.7099 461.86,84.7099 
 467.017,84.7099 472.174,84.7099 477.33,84.7099 482.487,84.7099 487.644,84.7099 492.801,84.7099 497.958,84.7099 503.115,84.7099 508.272,84.7099 513.429,84.7099 
 518.586,84.7099 523.743,84.7099 528.9,84.7099 534.057,84.7099 539.214,84.7099 544.371,84.7099 549.528,84.7099 554.685,84.7099 559.842,84.7099 564.999,84.7099 
 
 "/>
<polyline clip-path="url(#clip3302)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 54.4598,326.762 59.6167,326.762 64.7737,326.762 69.9306,326.762 75.0876,326.762 80.2446,326.762 85.4015,326.762 90.5585,326.762 95.7154,326.762 100.872,326.762 
 106.029,326.762 111.186,326.762 116.343,326.762 121.5,326.762 126.657,326.762 131.814,326.762 136.971,326.762 142.128,326.762 147.285,326.762 152.442,326.762 
 157.599,326.762 162.756,326.762 167.913,326.762 173.07,326.762 178.227,326.762 183.384,326.762 188.541,326.762 193.698,326.762 198.855,326.762 204.012,326.762 
 209.169,326.762 214.326,326.762 219.482,326.762 224.639,326.762 229.796,326.762 234.953,326.762 240.11,326.762 245.267,326.762 250.424,326.762 255.581,326.762 
 260.738,326.762 265.895,326.762 271.052,326.762 276.209,326.762 281.366,326.762 286.523,326.762 291.68,326.762 296.837,326.762 301.994,326.762 307.151,326.762 
 312.308,326.762 317.465,326.762 322.622,326.762 327.779,326.762 332.936,326.762 338.093,326.762 343.25,326.762 348.406,326.762 353.563,326.762 358.72,326.762 
 363.877,326.762 369.034,326.762 374.191,326.762 379.348,326.762 384.505,326.762 389.662,326.762 394.819,326.762 399.976,326.762 405.133,326.762 410.29,326.762 
 415.447,326.762 420.604,326.762 425.761,326.762 430.918,326.762 436.075,326.762 441.232,326.762 446.389,326.762 451.546,326.762 456.703,326.762 461.86,326.762 
 467.017,326.762 472.174,326.762 477.33,326.762 482.487,326.762 487.644,326.762 492.801,326.762 497.958,326.762 503.115,326.762 508.272,326.762 513.429,326.762 
 518.586,326.762 523.743,326.762 528.9,326.762 534.057,326.762 539.214,326.762 544.371,326.762 549.528,326.762 554.685,326.762 559.842,326.762 564.999,326.762 
 
 "/>
<polyline clip-path="url(#clip3302)" style="stroke:#ffa500; stroke-width:4; stroke-opacity:1; fill:none" points="
 54.4598,205.736 59.6167,205.736 64.7737,205.736 69.9306,205.736 75.0876,205.736 80.2446,205.736 85.4015,205.736 90.5585,205.736 95.7154,205.736 100.872,205.736 
 106.029,205.736 111.186,205.736 1

# Construct a region graph

Set some parameters

In [25]:
global gID = 1

numGPs = 5
numSums = 5
meanFunction = MeanZero();
kernelFunctions = [SE(log(5.0),log(1.0))];
kernelFunctions = [Mat12Iso(log(5.0), log(1.0)), Mat32Iso(log(5.0), log(1.0)), Mat52Iso(log(5.0), log(1.0))]
#kernelFunctions = [LinIso(log(5.0))]
kernelFunctions = [Mat12Iso(log(5.0), log(1.0)), LinIso(log(5.0))]
noise = -1.;

# put some priors on the parameters
#set_priors!(kernelFunction, [Normal(log(5.0),1.0), Normal(log(1.0),1.0)])

In [26]:
# split size
δ = 10

# data range
minX = 0
maxX = 60

(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos(δ, minX, maxX);
# set IDs for convenients
RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));
root1 = convertToSPN(rootRegion, gpRegions, X, y, meanFunction, kernelFunctions, noise; overlap = 5)

(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos(δ, minX+(δ/2), maxX+(δ/2));
# set IDs for convenients
RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));
root2 = convertToSPN(rootRegion, gpRegions, X, y, meanFunction, kernelFunctions, noise; overlap = 5)


Gaussian Process Sum Node [ID: 1177, 
	 w_prior: [0.2, 0.2, 0.2, 0.2, 0.2], 
	 w_posterior: [0.2, 0.2, 0.2, 0.2, 0.2]]

# Posterior Inference over SPN-GP

In [143]:
function spn_mcmc(spn::GPSumNode, gpNodes::Vector;
              sampler::Klara.MCSampler=Klara.HMC(),
              nIter::Int = 1000,
              burnin::Int = 0,
              thin::Int = 1)
    
    gps = map(n -> n.gp, gpNodes)
    
    function logpost(hyp::Vector{Float64})  #log-target
        
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target!(gp)
        end
        
        dirty!(spn)
        spn_update!(spn)
        return spn_posterior(spn)
    end

    function dlogpost(hyp::Vector{Float64}) #gradient of the log-target
        Kgrad_buffer = Array{Float64}(gp.nobsv, gp.nobsv)
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target_and_dtarget!(gp, noise=false, mean=false, kern=true)
            println(gp.dtarget)
        end
        
        dirty!(spn)
        spn_update!(spn)
        
        return gp.dtarget
    end
    
    start = reduce(vcat, map(gp -> get_params(gp)[2:3], gps))
    starting = Dict(:p=>start)
    q = BasicContMuvParameter(:p, logtarget=logpost, gradlogtarget=dlogpost) 
    model = likelihood_model(q, false)                               #set-up the model
    job = BasicMCJob(model, sampler, BasicMCRange(nsteps=nIter, thinning=thin, burnin=burnin), starting)   #set-up MCMC job
    print(job)                                             #display MCMC set-up for the user
    
    run(job)                          #Run MCMC
    chain = Klara.output(job)         # Extract chain
    #set_params!(gp,start)      #reset the parameters stored in the GP to original values
    return chain
end

spn_mcmc (generic function with 1 method)

In [144]:
#leafs = filter(n -> isa(n, GPLeaf), SumProductNetworks.getOrderedNodes(root))

In [145]:
#spn_mcmc(root, leafs)

## posterior inference of weights

In [27]:
spn_update!(root1)
spn_update!(root2)

-115.90451984730873

In [28]:
spn_posterior(root1)
spn_posterior(root2)

-112.977544807628

In [29]:
#dirty!(root)

In [30]:
root = GPSumNode(nextID(), Int[]);


for child in children(root1)
    add!(root, child)
end

for child in children(root2)
    add!(root, child)
end

fill!(root.prior_weights, 1. / length(root))
fill!(root.posterior_weights, 1. / length(root))

spn_update!(root)
spn_posterior(root)

-104.82441506160316

## Plotting

In [31]:
x = linspace(minimum(X),maximum(X),100)
r = Dict{Int, SPNGPResult}()
spn_predictIndep(root, x[1], r)

SPNGPResult(-0.009825895200987785, -0.009825895200987785, 0.018441837786811605)

In [15]:
function spn_predict(root, x::Vector)
    μ = zeros(length(x))
    σ = zeros(length(x))
    
    for (xi, xx) in enumerate(x)
        #println(xi)
        r = Dict{Int, SPNGPResult}()
        spn_predictIndep(root, xx, r)
        μi = r[root.id].mean
        #μ2i = r[root.id].meansqr
        σ2i = r[root.id].stdsqr
                
        μ[xi] = μi
        σ[xi] = σ2i #sqrt(σ2i + μ2i - μi^2)
    end
    
    return (μ, sqrt.(σ))
    
    #r = Dict{Int, Dict{Int, SPNGPResult}}()
    #spn_predict_moment(root, x, collect(1:length(x)), r)
    
    #return (r[root.id].mean, r[root.id].stdsqr)
end

function spn_density(root, x, y)
    r = Dict{Int, Float64}()
    spn_logdensityIndep(root, x, y, r)
    
    return exp(r[root.id])
end

spn_density (generic function with 1 method)

In [17]:
x = Float64.(collect(linspace(minimum(X),maximum(X),100)));

In [18]:
(μ, σ) = spn_predict(root, x)

([-3.04683e-7, -3.75468e-7, -4.46253e-7, -5.17038e-7, -5.87823e-7, -6.58607e-7, -7.29392e-7, -8.00177e-7, -8.70962e-7, -9.41747e-7  …  -1.42895e-5, -1.4441e-5, -1.45925e-5, -1.4744e-5, -1.48956e-5, -1.50471e-5, -1.51986e-5, -1.53501e-5, -1.55017e-5, -1.56532e-5], [0.135335, 0.135335, 0.135335, 0.135335, 0.135335, 0.135335, 0.135335, 0.135335, 0.135335, 0.135335  …  0.135341, 0.135341, 0.135341, 0.135341, 0.135341, 0.135342, 0.135342, 0.135342, 0.135342, 0.135342])

In [70]:
xx = linspace(minimum(X),maximum(X),S)
yy = linspace(-2,2,S)

Z = zeros(length(xx), length(yy))
for xi in 1:length(xx)
    for yi in 1:length(yy)
        Z[xi, yi] = spn_density(root, xx[xi], yy[yi])
    end
end

LoadError: [91mInterruptException:[39m

In [39]:
plot(xx, yy, Z')

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
<polyline clip-path="url(#clip4602)" style="stroke:#0d0829; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.2591,222.359 34.2193,222.359 39.1795,222.359 44.1396,222.36 49.0998,222.362 54.0599,222.362 59.0201,222.362 63.9803,222.361 68.9404,222.361 73.9006,222.362 
 78.8607,222.362 83.8209,222.362 88.7811,222.362 93.7412,222.362 95.0455,225.065 97.0827,228.742 98.7014,231.053 103.662,231.686 108.622,232.317 108.854,232.419 
 113.582,234.968 118.346,236.095 118.542,236.156 119.221,236.095 123.502,235.841 128.462,235.889 128.668,236.095 132.339,239.772 133.422,240.76 135.905,243.449 
 138.383,245.697 138.955,247.126 140.622,250.803 143.283,254.479 143.343,254.558 144.763,258.156 147.026,261.833 148.303,263.463 148.992,265.51 150.484,269.186 
 152.919,272.863 153.263,273.333 153.629,276.54 154.073,280.217 154.668,283.894 155.595,287.57 157.201,291.247 158.223,292.985 158.62,294.924 159.423,298.601 
 160.569,302.278 162.441,305.954 163.183,307.149 163.519,309.631 164.036,313.308 164.728,316.985 165.793,320.662 167.607,324.338 168.144,325.249 168.597,328.015 
 169.291,331.692 170.291,335.369 171.922,339.046 173.104,341.012 173.647,342.722 174.996,346.399 177.118,350.076 178.064,351.416 179.229,353.753 181.621,357.429 
 183.024,359.159 183.793,361.106 185.441,364.783 187.934,368.46 187.984,368.531 192.944,371.037 196.958,368.46 197.905,367.933 202.865,367.652 205.753,368.46 
 207.825,369.262 209.286,368.46 212.785,366.833 214.248,364.783 216.354,361.106 217.745,357.914 218.021,357.429 219.873,353.753 220.985,350.076 221.717,346.399 
 222.267,342.722 222.705,339.489 222.961,339.046 224.825,335.369 225.887,331.692 226.535,328.015 226.972,324.338 227.31,320.662 227.625,316.985 227.666,316.61 
 229.615,313.308 230.999,309.631 231.925,305.954 232.626,302.402 232.704,302.278 234.869,298.601 236.189,294.924 237.097,291.247 237.586,289.016 242.546,287.598 
 242.562,287.57 244.589,283.894 245.741,280.217 246.453,276.54 246.945,272.863 247.344,269.186 247.506,267.819 248.81,265.51 250.266,261.833 251.15,258.156 
 251.745,254.479 252.206,250.803 252.466,248.734 253.413,247.126 255.1,243.449 256.145,239.772 256.869,236.095 257.426,232.633 257.556,232.419 259.614,228.742 
 260.875,225.065 261.739,221.388 262.387,217.978 267.347,219.306 268.571,217.711 270.912,214.035 272.307,211.082 272.743,210.358 274.736,206.681 276.106,203.004 
 277.181,199.327 277.267,199.062 279.759,195.651 281.584,191.974 282.227,190.522 287.187,188.887 287.968,188.297 292.148,185.094 297.108,187.006 299.591,188.297 
 302.068,189.7 307.028,191.671 307.624,191.974 311.988,194.454 316.241,191.974 316.948,191.614 317.255,191.974 320.56,195.651 321.909,196.933 322.652,199.327 
 323.983,203.004 325.975,206.681 326.869,208.05 327.768,210.358 329.427,214.035 331.829,217.678 336.789,216.758 340.899,214.035 341.749,213.498 346.542,210.358 
 346.709,210.26 346.768,210.358 348.897,214.035 351.67,217.447 351.917,217.711 355.817,221.388 356.63,222.057 358.787,225.065 361.59,228.11 364.195,225.065 
 366.55,222.344 371.51,222.344 376.47,222.344 381.43,222.344 386.391,222.344 391.351,222.344 396.311,222.344 401.271,222.344 406.231,222.345 411.191,222.348 
 416.152,222.348 421.112,222.348 426.072,222.348 431.032,222.348 435.992,222.348 440.952,222.348 445.913,222.348 450.873,222.348 455.833,222.354 460.793,222.354 
 465.753,222.354 470.713,222.354 475.674,222.354 480.634,222.354 485.594,222.354 490.554,222.354 495.514,222.354 500.474,222.354 505.434,222.354 510.395,222.354 
 515.355,222.354 520.315,222.354 
 "/>
<polyline clip-path="url(#clip4602)" style="stroke:#270b52; stroke-width:1; stroke-opacity:1; fill:none" points="
 520.315,218.28 515.355,218.28 510.395,218.28 505.434,218.28 500.474,218.28 495.514,218.28 490.554,218.28 485.594,218.28 480.634,218.28 475.674,218.28 
 470.

In [71]:
xx = linspace(minimum(X),maximum(X),S)
yy = zeros(length(xx), 2)

for (i,x) in enumerate(xx)
    r = Dict{Int, SPNGParamResult}()
    spn_wavelength(root, x, r)
    yy[i, 1] = r[root.id].loglength
    yy[i, 2] = r[root.id].logsigma
    
    #r = Dict{Int, SPNGParamResult}()
    #spn_wavelength(root2, x, r)
    #yy[i, 2] = exp(r[root2.id].loglength)
end

#map(x -> spn_wavelength(root, x), xx);

In [75]:
p = plot(scatter(X, y, label = "observations", title = "Motorcycle", ylabel = "y", xlabel = "x"),
plot(xx, yy[:,1], label = ["wave-length"], ylabel = "log wave-length", xlabel = "x"),
plot(xx, yy[:,2], label = ["sigma"], ylabel = "log sigma", xlabel = "x"),
layout = (3, 1),
size = (700, 700)
)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 Motorcycle 
 
 
 x 
 
 
 y 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 observations 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 x 
 
 
 log wave-length 
 
<polyline clip-path="url(#clip9703)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.8916,429.075 61.209,429.075 67.5264,429.075 73.8438,429.075 80.1612,429.075 86.4787,429.075 92.7961,429.075 99.1135,429.075 105.431,429.075 111.748,429.075 
 118.066,429.075 124.383,429.075 130.701,429.075 137.018,429.075 143.335,376.069 149.653,376.069 155.97,376.069 162.288,376.069 168.605,376.069 174.922,376.069 
 181.24,376.069 187.557,376.069 193.875,376.069 200.192,376.079 206.509,376.079 212.827,376.079 219.144,376.079 225.462,376.079 231.779,376.079 238.096,376.079 
 244.414,376.079 250.731,376.079 257.049,391.087 263.366,391.087 269.683,391.087 276.001,391.087 282.318,391.087 288.636,391.087 294.953,391.087 301.271,391.087 
 307.588,391.087 313.905,391.075 320.223,391.075 326.54,391.075 332.858,391.075 339.175,391.075 345.492,391.075 351.81,391.075 358.127,391.075 364.445,391.075 
 370.762,424.128 377.079,424.128 383.397,424.128 389.714,424.128 396.032,424.128 402.349,424.128 408.666,424.128 414.984,424.128 421.301,424.128 427.619,424.212 
 433.936,424.212 440.253,424.212 446.571,424.212 452.888,424.212 459.206,424.212 465.523,424.212 471.841,424.212 478.158,424.212 484.475,340.466 490.793,340.466 
 497.11,340.466 503.428,340.466 509.745,340.466 516.062,340.466 522.38,340.466 528.697,340.466 535.015,340.466 541.332,340.449 547.649,340.449 553.967,340.449 
 560.284,340.449 566.602,340.449 572.919,340.449 579.236,340.449 585.554,340.449 591.871,340.449 598.189,258.332 604.506,258.332 610.823,258.332 617.141,258.332 
 623.458,258.332 629.776,258.332 636.093,258.332 642.411,258.332 648.728,258.332 655.045,258.268 661.363,258.268 667.68,258.268 673.998,258.268 680.315,258.268 
 
 "/>
 
 
 
 
 wave-length 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 x 
 
 
 log sigma 
 
<polyline clip-path="url(#clip9704)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.8916,485.039 61.209,485.039 67.5264,485.039 73.8438,485.039 80.1612,485.039 86.4787,485.039 92.7961,485.039 99.1135,485.039 105.431,485.039 111.748,485.039 
 118.066,485.039 124.383,485.039 130.701,485.039 137.018,485.039 143.335,643.843 149.653,643.843 155.97,643.843 162.288,643.843 168.605,643.843 174.922,643.843 
 181.24,643.843 187.557,643.843 193.875,643.843 200.192,643.854 206.509,643.854 212.827,643.854 219.144,643.854 225.462,643.854 231.779,643.854 238.096,643.854 
 244.414,643.854 250.731,643.854 257.049,644.272 263.366,644.272 269.683,644.272 276.001,644.272 282.318,644.272 288.636,644.272 294.953,644.272 301.271,644.272 
 307.588,644.272 313.905,644.269 320.223,644.269 326.54,644.269 332.858,644.269 339.175,644.269 345.492,644.269 351.81,644.269 358.127,644.269 364.445,644.269 
 370.762,655.846 377.079,655.846 383.397,655.846 389.714,655.846 396.032,655.846 402.349,655.846 408.666,655.846 414.984,655.846 421.301,655.846 427.619,651.774 
 433.936,651.774 440.253,651.774 446.571,651.774 452.888,651.774 459.206,651.774 465.523,651.774 471.841,651.774 478.158,651.774 484.475,485.11 490

In [76]:
savefig("../plots/MotorCycle_hyper-parameters.png")

In [32]:
for gpnode in unique(filter(n -> isa(n, GPLeaf), SumProductNetworks.getOrderedNodes(root)))
    println(get_params(gpnode.gp.k))
    iparams = copy(get_params(gpnode.gp.k))
    println(pointer_from_objref(gpnode.gp))
    #println(gpnode.gp.X)
    res = optimize!(gpnode.gp, mean = false, kern = true, noise = false, lik=false)
    
    p = get_params(gpnode.gp.k)
    
    lowerBound = -10
    upperBound = 10
    
    p[p .> upperBound] = upperBound
    p[p .< lowerBound] = lowerBound
    
    set_params!(gpnode.gp.k, p)
    update_target!(gpnode.gp)
    println(gpnode.gp.k)
end

[1.60944, 0.0]
Ptr{Void} @0x000000012af49f90
Type: GaussianProcesses.Mat12Iso, Params: [10.0, -10.0]

[1.60944, 0.0]
Ptr{Void} @0x000000012af4a910
Type: GaussianProcesses.Mat12Iso, Params: [3.19373, -0.169754]

[1.60944, 0.0]
Ptr{Void} @0x000000012af4a810
Type: GaussianProcesses.Mat12Iso, Params: [2.5909, -0.196529]

[1.60944, 0.0]
Ptr{Void} @0x000000012af4a590
Type: GaussianProcesses.Mat12Iso, Params: [1.26069, -1.02177]

[1.60944, 0.0]
Ptr{Void} @0x000000012af4a410
Type: GaussianProcesses.Mat12Iso, Params: [10.0, -10.0]

[1.60944, 0.0]
Ptr{Void} @0x000000012af4a090
Type: GaussianProcesses.Mat12Iso, Params: [10.0, -10.0]

[1.60944]
Ptr{Void} @0x000000012af4a110
Type: GaussianProcesses.LinIso, Params: [10.0]

[1.60944]
Ptr{Void} @0x000000012af4a490
Type: GaussianProcesses.LinIso, Params: [10.0]

[1.60944]
Ptr{Void} @0x000000012af4a610
Type: GaussianProcesses.LinIso, Params: [6.11206]

[1.60944]
Ptr{Void} @0x000000012af4a890
Type: GaussianProcesses.LinIso, Params: [3.97323]

[1.60944]
P

In [69]:
S = 100

#x = linspace(minimum(X),maximum(X),S)
#xx = reduce(hcat, x for k in 1:S)
#yy = reduce(hcat, spn_randIndep(root, x) for k in 1:S)

x = Float64.(collect(linspace(minimum(X),maximum(X),S)));
(μ, σ) = spn_predict(root, x)

gp = GP(reshape(X, 1, length(X)), y, MeanZero(), SE(log(5.0),log(1.0)), -1.)
μgp, σgp = predict_y(gp,linspace(minimum(X),maximum(X),S));

plt = scatter(X, y, label = "observations", size = (1024, 768), title = "Motorcycle - GP vs. SPN-P with GP leaves ")
#scatter!(xx+(rand(size(xx)) * 0.3), yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");



plot!(linspace(minimum(X),maximum(X),S), w =4, μgp + 2*σgp, color = :lightgreen, label = "classical GP: mean + 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp - 2*σgp, color = :lightgreen, label = "classical GP: mean - 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp, color = :green, label = "classical GP: mean")


#plot!(linspace(minimum(X),maximum(X),S), w =2, μ + 2*σ, color = :green, label = "SPN GP mean + 2*std")
#plot!(linspace(minimum(X),maximum(X),S), w =2, μ - 2*σ, color = :green, label = "SPN GP mean - 2*std")
#plot!(linspace(minimum(X),maximum(X),S), w =2, μ, color = :red, label = "SPN GP mean")

#vline!([minimum(map(r -> r.min, gpRegions))], c = :orange)
#vline!(map(r -> r.max, gpRegions), c = :orange)

plot!(xx, yy, Z', label = "GP-SPN")
#plot!(linspace(minimum(X),maximum(X),S), linspace(-2,2,S), (x, y) -> spn_density(root, x, y))

plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Motorcycle - GP vs. SPN-P with GP leaves 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip9102)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 187.555,592.229 194.302,593.529 201.05,594.295 207.798,594.789 214.545,595.211 221.293,595.676 228.041,596.213 234.788,596.775 241.536,597.268 248.284,597.582 
 255.031,597.619 261.779,597.319 268.527,596.675 275.274,595.742 282.022,594.636 288.77,593.532 295.517,592.65 302.265,592.236 309.012,592.539 315.76,593.789 
 322.508,596.171 329.255,599.805 336.003,604.739 342.751,610.933 349.498,618.263 356.246,626.526 362.994,635.446 369.741,644.685 376.489,653.863 383.237,662.575 
 389.984,670.416 396.732,677.01 403.48,682.036 410.227,685.25 416.975,686.499 423.722,685.732 430.47,682.992 437.218,678.407 443.965,672.178 450.713,664.554 
 457.461,655.824 464.208,646.293 470.956,636.276 477.704,626.081 484.451,616.005 491.199,606.32 497.947,597.272 504.694,589.073 511.442,581.895 518.19,575.874 
 524.937,571.099 531.685,567.616 538.432,565.42 545.18,564.456 551.928,564.62 558.675,565.76 565.423,567.69 572.171,570.197 578.918,573.064 585.666,576.08 
 592.414,579.055 599.161,581.83 605.909,584.282 612.657,586.32 619.404,587.894 626.152,588.986 632.9,589.617 639.647,589.839 646.395,589.738 653.142,589.422 
 659.89,589.011 666.638,588.627 673.385,588.377 680.133,588.345 686.881,588.583 693.628,589.109 700.376,589.908 707.124,590.936 713.871,592.128 720.619,593.403 
 727.367,594.671 734.114,595.846 740.862,596.851 747.61,597.625 754.357,598.13 761.105,598.35 767.852,598.295 774.6,597.992 781.348,597.48 788.095,596.807 
 794.843,596.023 801.591,595.167 808.338,594.269 815.086,593.336 821.834,592.344 828.581,591.236 835.329,589.911 842.077,588.231 848.824,586.03 855.572,583.131 
 
 "/>
<polyline clip-path="url(#clip9102)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 187.555,630.922 194.302,629.863 201.05,629.596 207.798,629.847 214.545,630.369 221.293,630.965 228.041,631.495 234.788,631.865 241.536,632.023 248.284,631.941 
 255.031,631.609 261.779,631.033 268.527,630.24 275.274,629.283 282.022,628.25 288.77,627.264 295.517,626.49 302.265,626.122 309.012,626.377 315.76,627.474 
 322.508,629.617 329.255,632.969 336.003,637.628 342.751,643.602 349.498,650.802 356.246,659.031 362.994,667.998 369.741,677.335 376.489,686.625 383.237,695.438 
 389.984,703.353 396.732,709.996 403.48,715.053 410.227,718.289 416.975,719.554 423.722,718.792 430.47,716.036 437.218,711.408 443.965,705.104 450.713,697.382 
 457.461,688.548 464.208,678.931 470.956,668.867 477.704,658.683 484.451,648.681 491.199,639.134 497.947,630.276 504.694,622.297 511.442,615.348 518.19,609.535 
 524.937,604.921 531.685,601.529 538.432,599.343 545.18,598.31 551.928,598.348 558.675,599.344 565.423,601.16 572.171,603.63 578.918,606.565 585.666,609.76 
 592.414,612.999 599.161,616.078 605.909,618.816 612.657,621.073 619.404,622.765 626.152,623.866 632.9,624.412 639.647,624.487 646.395,624.215 653.142,623.739 
 659.89,623.209 666.638,622.768 673.385,622.539 680.133,622.614 686.881,623.05 693.628,623.861 700.376,625.016 707.124,626.452 713.871,628.073 720.619,629.768 
 727.367,631.426 734.114,632.941 740.862,634.228 747.61,635.227 754.357,635.903 761.105,636.249 767.852,636.278 774.6,636.024 781.348,635.529 788.095,634.846 
 794.843,634.026 801.591,633.12 808.338,632.178 815.086,631.254 821.834,630.414 828.581,629.748 835.329,629.376 842.077,629.448 848.824,630.14

In [41]:
savefig("../plots/MotorCycle_GP_vs_SPN_without_overlap_lin_matern12.png")

## Histogram over parameters

## Graph Plotting

In [44]:
sources = Int[]
destinations = Int[]

# add edges from regions to partitions
for r in filter(r -> isa(r, SumRegion), union(sumRegions, gpRegions))
    for p in r.partitions
        push!(sources, RegionIDs[r])
        push!(destinations, PartitionIDS[p])
    end
end

# add edges from partitions to regions
for p in allPartitions
    for r in p.regions
        push!(sources, PartitionIDS[p])
        push!(destinations, RegionIDs[r])
    end
end

In [45]:
nodenames = vcat(map(r -> string("+"),  union(sumRegions, gpRegions)), map(r -> string("*"), allPartitions));
colorings = vcat(map(r -> isa(r, SumRegion) ? :lightblue : :lightgreen,  union(sumRegions, gpRegions)), map(r -> :black, allPartitions));

In [47]:
#gr(size = (400, 400))
#graphplot(sources, destinations, names = nodenames, method = :tree, fontsize = 7, nodesize = 3, m = colorings, nodeshape = :rect)